Helpers: 
* https://stackoverflow.com/questions/32173053/copy-columns-from-workbook-paste-in-second-sheet-of-second-workbook-openpyxl
* https://automatetheboringstuff.com/chapter12/

In [1]:
import sys, os, openpyxl, xlrd
from openpyxl.utils import get_column_letter
from copy import copy
import pandas as pd
import numpy as np

In [2]:
#Get the filename
dataDir = '../../Data'
dataFN = dataDir + os.sep + 'LouisianaWaterUse_Distribute.xlsx'
xlFN = dataDir + os.sep + 'LA_SWUDS_BalanceSheet1.xlsx'
outFN = dataDir + os.sep + 'LA_SWUDS_BalanceSheet2.xlsx'

In [3]:
#Read the data xlsx file in as a dataframe
dfRaw = pd.read_excel(dataFN,sheetname='data1')

In [4]:
#Select only 2000, 2005, and 2010 records and list the number of data for each column
dfSelect = dfRaw[dfRaw.Year.isin([2000,2005,2010])]

In [5]:
dfSelect.columns.values

array([u'County Code', u'SIC1', u'Description', u'Water Use Type',
       u'MP ID', u'MP Type', u'MP Name', u'Source Water Type',
       u'Aquifer Code', u'Aquifer Name2', u'Aquifer Group ID',
       u'Aquifer Group Name', u'HUC-8', u'HUC-12', u'Year',
       u'Annual Amt. (MGD)'], dtype=object)

In [6]:
#Pivot on year, listing usage (Annual Amt. MGD) by Source Water  Type and SCI1 Description
dfPivot = dfSelect.pivot_table(columns='Year',index=('SIC1','Description'),values='Annual Amt. (MGD)',aggfunc='sum')
dfPivot.reset_index(inplace=True)

In [7]:
#Create a list of sectors
sectorNames = dfPivot.Description.values.tolist()

In [8]:
#Open the workbook
wb = openpyxl.load_workbook(xlFN)

In [9]:
#Get the sheet
ws = wb.get_sheet_by_name('2000')

In [10]:
#Update the name of the first sector columns
ws['B2'] = sectorNames[0]
ws['B19'] = sectorNames[0]

In [11]:
#Loop through each sector, by index
for idx in range(1,len(sectorNames)):
    #get the sector at the index
    sectorName = sectorNames[idx]
    #get the letter of the column to create
    colLetter = get_column_letter(idx+1)
    #Add a new column
    for src, dst in zip(ws['B:B'], ws['{0}:{0}'.format(colLetter)]):
        dst.value = src.value
        if src.has_style:
            dst.font = copy(src.font)
            dst.fill = copy(src.fill)
            dst.number_format = copy(src.number_format)
            dst.alignment = copy(src.alignment)
            dst.border = copy(src.border)
    #Update the sector name
    ws['{}2'.format(colLetter)] = sectorName
    ws['{}19'.format(colLetter)] = sectorName

In [12]:
#Save
wb.save(outFN)